## Pre-Training [`GPT-Neo`](https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt_neo/modeling_gpt_neo.py) & [`RoBERTa`](https://github.com/huggingface/transformers/blob/main/src/transformers/models/roberta/modeling_roberta.py) on [`TinyStories`](https://huggingface.co/datasets/roneneldan/TinyStories)

This script shows how to pre-train both models. I put them in one notebook because the majority of the code is shared; but you may want to separate the logic per model. 

In [1]:
# %pip install torch wandb transformers[torch] datasets tqdm 
%load_ext autoreload
%autoreload 2 

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # or "0,1" for multiple GPUs

import wandb; wandb.login()
from transformers import (
    RobertaForMaskedLM, RobertaConfig, RobertaTokenizerFast,
    GPTNeoForCausalLM, GPTNeoConfig, GPT2TokenizerFast
)

from custom_gpt_gqa import CustomGPTNeoForCausalLM, GPTNeoGQASelfAttention

wandb: Currently logged in as: st1p42 (tiny-transformers). Use `wandb login --relogin` to force relogin


#### Models 
We consider GPT-Neo and BERT as base transformer architectures. This consists of the following blocks linked via residual connections:

- Embeddings: Map one-hot (sparse) token vectors to a dense vector of length `hidden_size`. Add positional encoding. 
- $n$ Blocks: Contain self-attention and FFNs.
- Head: Map hidden state back to a useful output. 

This specifies some of the model-related hyperparameters. I chose them based on what achieved reasonable performance in the [TinyStories paper](https://arxiv.org/abs/2305.07759), while also being feasible to train on our limited compute budgets. 

In [3]:
embed_size = 512
kqv_factor = 2
kqv_size = embed_size // kqv_factor
ffn_original_width = 1024
ffn_new_width = (embed_size // (2 * kqv_factor)) * (4 * kqv_factor - 4) + ffn_original_width

config_gpt = dict(

    # EMBEDDING PARAMETERS
    vocab_size              = 10_000,   # number of tokens in the vocabulary 
    hidden_size             = embed_size,      # embedding size (vector length) of each token
    max_position_embeddings = 512,      # maximum sequence length (context window)

    # BLOCKS (ATTN & FFN)
    num_layers          = 2,                    # number of transformer blocks
    attention_types     = [[["global", "local"], 1]], # (GPT-Neo-specific) global and local attention 
    num_heads           = 4,                    # attention heads
    window_size         = 256,                  # (GPT-Neo-specific) for local attention 
    intermediate_size   = 1024,                 # size of 'up-projection' layer in FFN

    pad_token_id = 0,           # need to specify this for tokenizer interop between models
)


config_gqa = dict(

    # EMBEDDING PARAMETERS
    vocab_size              = 10_000,   # number of tokens in the vocabulary
    hidden_size             = embed_size,      # embedding size (vector length) of each token
    max_position_embeddings = 512,      # maximum sequence length (context window)

    # BLOCKS (ATTN & FFN)
    num_layers          = 2,                    # number of transformer blocks
    attention_types     = [[["global", "local"], 1]], # (GPT-Neo-specific) global and local attention
    num_heads           = 4,                    # attention heads
    window_size         = 256,                  # (GPT-Neo-specific) for local attention
    intermediate_size   = ffn_new_width,                 # size of 'up-projection' layer in FFN

    pad_token_id = 0,           # need to specify this for tokenizer interop between models
)

config_rob = dict(
    
    # EMBEDDING PARAMETERS
    vocab_size              = 10_000,   
    hidden_size             = 512,
    # we add 1 as RoBERTa uses a special position embedding for the padding token (zero vector)
    max_position_embeddings = config_gpt['max_position_embeddings'] + 1,

    # BLOCKS (of course naming is different in roberta :) )
    num_hidden_layers = config_gpt['num_layers'],
    num_attention_heads = config_gpt['num_heads'],
    intermediate_size=1024,                     

    pad_token_id = 0,
)

config_gpt = GPTNeoConfig(**config_gpt)
config_rob = RobertaConfig(**config_rob)
config_gqa = GPTNeoConfig(**config_gqa)

In [33]:
from custom_gpt_gqa import CustomGPTNeoForCausalLM, GPTNeoGQASelfAttention

# TODO: you should set all pytorch & huggingface seeds here as model initialisation depends on it

gpt = GPTNeoForCausalLM(config=config_gpt)
rob = RobertaForMaskedLM(config=config_rob)

gqa_model_gpt = CustomGPTNeoForCausalLM(config_gqa=config_gqa)
# print(gqa_model_gpt.state_dict())
gqa_model_gpt.set_kqv_size(kqv_size)
gqa_model_gpt.set_attention()


print(f'''
    The custom model has {gqa_model_gpt.num_parameters():,} parameters,
    GPT Neo - {gpt.num_parameters():,} parameters,
    and RoBERTa - {rob.num_parameters():,} parameters.
    ''')

# gpt, rob # uncomment to see model architecture
# gqa_model_gpt

OrderedDict([('transformer.wte.weight', tensor([[-0.0374,  0.0040, -0.0319,  ..., -0.0062, -0.0067,  0.0224],
        [-0.0268,  0.0263, -0.0160,  ..., -0.0070,  0.0012, -0.0093],
        [-0.0100,  0.0212,  0.0238,  ..., -0.0335,  0.0009,  0.0068],
        ...,
        [-0.0301, -0.0007,  0.0210,  ...,  0.0118, -0.0189,  0.0175],
        [-0.0163,  0.0130, -0.0261,  ...,  0.0053,  0.0157, -0.0009],
        [ 0.0012,  0.0230, -0.0068,  ...,  0.0215,  0.0087, -0.0081]])), ('transformer.wpe.weight', tensor([[-0.0119, -0.0165, -0.0374,  ...,  0.0092,  0.0258,  0.0051],
        [ 0.0137, -0.0099,  0.0137,  ...,  0.0025, -0.0448, -0.0203],
        [ 0.0277,  0.0048,  0.0141,  ...,  0.0052, -0.0087,  0.0040],
        ...,
        [-0.0245,  0.0054, -0.0246,  ...,  0.0157, -0.0024, -0.0062],
        [ 0.0112, -0.0278,  0.0376,  ...,  0.0253, -0.0095, -0.0025],
        [ 0.0102, -0.0103,  0.0440,  ...,  0.0131, -0.0171, -0.0078]])), ('transformer.h.0.ln_1.weight', tensor([1., 1., 1., 1., 1., 1

#### Pre-Processing
All experiments use the same tokenizer, so in theory, we only need to preprocess our data once. This means you can subsequently skip this CPU-intensive task on the DelftBlue GPU nodes. 
(except if you are experimenting with different context lengths, in that case you will need to re-tokenise to recover potentially truncated inputs; see below).

When running experiments on delftblue, it may be fastest to load (parts of) the dataset into memory, as the cluster has pretty slow IO. See `datasets`, specifically [`load_dataset`'s `keep_in_memory` attribute](https://huggingface.co/docs/datasets/en/package_reference/loading_methods#datasets.load_dataset). 

In [5]:
import tqdm 
from datasets import load_dataset, DatasetDict

debug_mode = True

dataset = load_dataset('roneneldan/tinystories', num_proc=16)
small_dataset = DatasetDict({
    'train': dataset['train'].select(range(1000)),
    'validation': dataset['validation'].select(range(100))
})

if debug_mode:
    dataset = small_dataset

n_words = 0 
for story in tqdm.tqdm(dataset['train']['text']):    # tqdm is used for progress bars around iterables
    n_words += len(story.split())

print(f'''
    Train dataset has {len(dataset['train']['text']):,} samples, 
    totalling {n_words:,} words (avg. {n_words/len(dataset['train']['text']):,.1f} per story).
    ''')

Repo card metadata block was not found. Setting CardData to empty.
100%|██████████| 1000/1000 [00:00<00:00, 86484.06it/s]


    Train dataset has 1,000 samples, 
    totalling 183,801 words (avg. 183.8 per story).
    


#### Tokenisation
Two things happen during tokenisation, of which the latter is optional:

1. Map (sub-)words to integers $ \in [0, 10000]$.
2. Truncate/pad each input as necessary to fit within `hidden_size`. 

All it does is map frequent character strings (i.e. tokens) to integers, which it can then use to look up the corresponding embedding vector for that integer index.

The intuition is that by assigning each token its own embedding (vector of `1 x hidden_size`) in the transformer, will help it learn semantical understanding of these tokens. E.g. say that you have two tokens 'about' and 'around', the transformer should learn similar embedding vectors for each.

In [6]:
# NOTE: uncomment to tokenize the dataset (both tokenizers should give identical results)

# tokenized_gpt = dataset.map(
#     lambda x: tok_gpt(x['text'], truncation=True, padding='max_length'),
#     batched=True, num_proc=64, batch_size=1_000)                 # change num_proc to 1 if multithread issues
#
# tokenized_gpt.save_to_disk('./tokenized_dataset', num_proc=5)

# sanity check that the tokenisers are indeed identical
# tokenized_rob = dataset.map(
#     lambda x: tok_rob(x['text'], truncation=True, padding='max_length'),
#     batched=True, num_proc=64, batch_size=1_000)                 # change num_proc to 1 if multithread issues
#
# for split in ['train', 'validation']:
#     for sample_gpt, sample_rob in tqdm.tqdm(zip(tokenized_gpt[split], tokenized_rob[split])):
#         # print("GPT sample: ", sample_gpt)
#         print("GPT sample: ", sample_gpt)
#         print("ROBERTA sample: ", sample_rob)
#         assert sample_gpt == sample_rob

In [7]:
from transformers import PreTrainedTokenizer, PretrainedConfig

def get_tokenizer_for_config(Tok: PreTrainedTokenizer, config: PretrainedConfig):

    tokenizer = Tok.from_pretrained(
        '10k-tok',                                         # our custom tokenizer
        model_max_length=config.max_position_embeddings    # sequence length (context window)
    )

    # we're using our special tokenizer with only 10'000 tokens instead of 50'256
    assert tokenizer.vocab_size == config.vocab_size

    print(f'padding token is {tokenizer.pad_token}')
    print(f'padding token in config: {config.pad_token_id}, in tokeniser: {tokenizer.pad_token_id}')

    return tokenizer

tok_gpt = get_tokenizer_for_config(GPT2TokenizerFast, config_gpt)
tok_rob = get_tokenizer_for_config(RobertaTokenizerFast, config_rob)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'GPT2TokenizerFast'.


padding token is <pad>
padding token in config: 0, in tokeniser: 0
padding token is <pad>
padding token in config: 0, in tokeniser: 0


In [8]:
tokenized_gpt = dataset.map(
    lambda x: tok_gpt(x['text'], truncation=True, padding='max_length'),
    batched=True, num_proc=64, batch_size=1_000)                 # change num_proc to 1 if multithread issues

tokenized_gpt.save_to_disk('./tokenized_dataset', num_proc=5)

Saving the dataset (0/5 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/5 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

In [9]:
from datasets import load_from_disk 
tokenized_dataset = load_from_disk(f'./tokenized_dataset')

train_dataset = tokenized_dataset['train']
eval_dataset  = tokenized_dataset['validation']

assert len(tokenized_dataset['train'][0]['input_ids']) == config_gpt.max_position_embeddings
tokenized_dataset['train'][0]['input_ids'][-10:]
# should be pad tokens (0), given that most short stories are <512 tokens

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

#### Training
Before we get started, you may want to specify which GPU to use. See the first cell in this notebook; make sure to run it before anything else. 

Huggingface provides some powerful (and often confusingly long) APIs for model training. The `TrainingArguments` specifies our hyperparameters, which are used by the `Trainer` taking in the remaining objects (like `model`, `tokenizer`, and `train_dataset`). Specifically:

- `learning_rate` and `num_train_epochs` determine how much the model learns. A higher rate is faster, but more unstable. More epochs (entire passes over the dataset) yields incrementally better results, at the cost of more training time. 
- Batch sizes determine how many samples the model sees in *parallel*. Given `gradient_accumulation_steps=1` and a `batch_size=8`, the model will backpropagate the average loss of 8 samples; if `batch_size=1`, it will average the loss of `gradient_accumulation_steps` samples. It is important to make sure the backpropagated loss is averaged over the same number of samples, when comparing models. 

- `data_collator` batches (and optionally, pads) the input for the model. We have already padded in our `tokenized_dataset`, and leaving this argument empty will automatically batch the inputs. So why do we need it? 

    Glad you asked. This has to do with how the loss is computed in causal language modelling. In our case, we try to predict $p(y | x)$, where $x$ is an input sequence of tokens, and $y$ is the next token following that sequence. Our model, unaware of the target token $y$, outputs $\hat y$. 
    
    For `Trainer` to compute the (cross-entropy) loss, we need to provide it with both $y$ and $\hat y$. The `DataCollatorForLanguageModeling` knows this, and provides the next token $y$ as a separate part of the input, to the `Trainer`.

    The loss is the backbone of backpropagation, which we need to actually improve our model. If this is confusing, please re-watch Karpathy's GPT tutorial. 

If you prefer to write the training loop yourself, check out HF' `run_clm_no_trainer.py` scripts. (`run_mlm_no_trainer.py` for RoBERTa-style masked-language modelling, as opposed to causal language modelling). This can be useful to give you better control over which devices are used for training. 

In [10]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

def get_hyperparameters(model, dataset):
    ''' common hyperparameters to give to the trainer '''

    # TRAINING HYPERPARAMETERS 
    batch_size = 4                  # TinyStories uses 80, but I am training locally on my poor M1 Air
    num_train_epochs = 1             # TinyStories doesn't mention
    gradient_accumulation_steps = 16 # TinyStories uses 16

    lr = 5e-4                        # TinyStories uses 5e-4, higher values better for small models

    # future you will thank you for descriptive model names
    # TODO: customise this name such that every model you train has a unique identifier!
    config      = model.config 
    model_name  = '-'.join([
        'CUSTOM-GQA' if isinstance(model, CustomGPTNeoForCausalLM) else 'GPT' if isinstance(model, GPTNeoForCausalLM) else 'BERT',
        f'{model.num_parameters()//1e6:.1f}M',
        f'{config.num_layers if isinstance(model, GPTNeoForCausalLM) else config.num_hidden_layers}L', 
        f'{config.num_heads if isinstance(model, GPTNeoForCausalLM) else config.num_attention_heads}H', 
        f'{config.hidden_size}C',
        f'{config.intermediate_size}I'
    ])

    _train_steps = len(dataset) // (batch_size * gradient_accumulation_steps)
    eval_steps = _train_steps // 10 # evaluate every 10% of training steps

    return dict(
        model_name = model_name,
        batch_size = batch_size, 
        num_train_epochs = num_train_epochs,
        gradient_accumulation_steps = gradient_accumulation_steps,
        lr = lr,
        eval_steps = eval_steps
    )

params_gpt = get_hyperparameters(gpt, train_dataset)
params_rob = get_hyperparameters(rob, train_dataset)
params_gqa = get_hyperparameters(gqa_model_gpt, train_dataset)

In [11]:
def get_trainer(
        model, tokenizer, train_dataset, eval_dataset, output_dir,
        model_name, batch_size, num_train_epochs, gradient_accumulation_steps, lr, eval_steps):
    ''' more general training arguments you likely want to keep fixed'''

    training_args = TrainingArguments(

        seed       = 42,
        use_cpu    = False, # use GPU if available (not necessarily faster on laptops, but Apple's MPS have good support)

        output_dir = os.path.join(output_dir, model_name),

        # NOTE: training params
        learning_rate    = lr,
        num_train_epochs = num_train_epochs,
        # Use a smaller batch size to fit into GPU RAM. 
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size  = batch_size,
        # You should aim to have the same amount of samples per acc step, in all of your experiments!
        # so, if you increase batch_size, decrease gradient_accumulation_steps by the same factor.
        gradient_accumulation_steps = gradient_accumulation_steps,

        # NOTE: Evaluation params
        # wandb is great for tracking experiments, it will even (try to) save your code nowadays
        evaluation_strategy = 'steps',
        eval_steps = eval_steps,
        save_steps = eval_steps,

        logging_first_step=True,
        logging_steps=100,
        report_to  = 'wandb',
    )

    trainer = Trainer(
        model = model, 
        args = training_args, 
        train_dataset = train_dataset, 
        eval_dataset = eval_dataset,
        data_collator = DataCollatorForLanguageModeling(
            tokenizer, mlm=isinstance(model, RobertaForMaskedLM)),
    )

    # print amount of training steps, and how often the model is evaluated
    print(f'''
    Retrieving Trainer for \033[1m{model_name}\033[0m
        training for {num_train_epochs} epochs, {len(train_dataset)} samples
        {batch_size} batch size, {gradient_accumulation_steps} accumulation steps
        gives {len(train_dataset)//(batch_size * gradient_accumulation_steps)} training steps.
        Evaluating every {eval_steps} steps, {len(eval_dataset)} samples 
        ''')

    return trainer

In [12]:
out_dir = './results/models_baseline/' 

trainer_gpt = get_trainer(gpt, tok_gpt, train_dataset, eval_dataset, out_dir, **params_gpt)
trainer_rob = get_trainer(rob, tok_rob, train_dataset, eval_dataset, out_dir, **params_rob)

trainer_gqa_gpt = get_trainer(gqa_model_gpt, tok_gpt, train_dataset, eval_dataset, out_dir, **params_gqa)


    Retrieving Trainer for GPT-9.0M-2L-4H-512C-1024I
        training for 1 epochs, 1000 samples
        4 batch size, 16 accumulation steps
        gives 15 training steps.
        Evaluating every 1 steps, 100 samples 
        

    Retrieving Trainer for BERT-9.0M-2L-4H-512C-1024I
        training for 1 epochs, 1000 samples
        4 batch size, 16 accumulation steps
        gives 15 training steps.
        Evaluating every 1 steps, 100 samples 
        

    Retrieving Trainer for CUSTOM-GQA-9.0M-2L-4H-512C-1536I
        training for 1 epochs, 1000 samples
        4 batch size, 16 accumulation steps
        gives 15 training steps.
        Evaluating every 1 steps, 100 samples 
        


Finally, we can train. 

This configuration takes ≤24hr to pre-train on my M1 Macbook Air with 16GB RAM. Python takes ≤4GB VRAM at a `batch_size=16` and ≤11GB at `batch_size=64`, though they take the same amount of time to train - likely because this processor is not designed to move that much data in and out of RAM constantly. And tbh, the GPU be lacking. If you decide to go the local-training-route, consider [chai](https://github.com/lvillani/chai) to keep your (Apple) laptop awake – there's probably a windows/linux equivalent too. 

In [13]:
def do_train(trainer: Trainer, name: str, out_dir: str):

    wandb.init(project='tiny-transformers', name=name, group='baseline', config=trainer.args)
    # wandb.init(project='tinystories', name=name, config=trainer.args)
    trainer.train()
    trainer.save_model(os.path.join(out_dir, name))
    wandb.finish()

In [18]:
# words vs. tokens 
len(train_dataset['text'][11]), len(train_dataset[11]['input_ids'])
print(params_gqa['model_name'])
trainer_gqa_gpt.model

CUSTOM-GQA-9.0M-2L-4H-512C-1536I


CustomGPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(10000, 512)
    (wpe): Embedding(512, 512)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-1): 2 x GPTNeoBlock(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoGQASelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=512, out_features=256, bias=False)
            (v_proj): Linear(in_features=512, out_features=256, bias=False)
            (q_proj): Linear(in_features=512, out_features=256, bias=False)
            (out_proj): Linear(in_features=256, out_features=512, bias=True)
          )
        )
        (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=512, out_features=1536, bias=True)
          (c_proj): Linear

In [19]:
# do_train(trainer_rob, params_rob['model_name'], out_dir)

In [20]:

do_train(trainer_gqa_gpt, params_gqa['model_name'], out_dir)

Step,Training Loss,Validation Loss


eval/loss,█▇▆▅▄▄▃▃▂▂▂▁▁▁▁
eval/runtime,▁▁▁▅██▇▇▇▇▇▇▇▇▇
eval/samples_per_second,███▃▁▁▁▂▂▁▂▂▂▂▂
eval/steps_per_second,███▃▁▁▁▂▂▁▂▂▂▂▂
train/epoch,▁▁▂▂▃▃▄▄▅▅▅▆▇▇███
train/global_step,▁▁▁▂▃▃▃▄▅▅▅▆▇▇▇██
train/grad_norm,▁
train/learning_rate,▁
train/loss,▁
eval/loss,6.42016
eval/runtime,25.8488


In [36]:
import torch
model_name = 'CUSTOM-GQA-9.0M-2L-4H-512C-1536I'
trained_model = trainer_gqa_gpt.model
torch.save(trained_model.state_dict(), f'results/models_baseline/{model_name}/model_state.pt')

In [21]:
wandb.finish()

#### Using your Pre-Trained Model 
Try out your own model on some prompts!

In [22]:
# I made a small function for pretty printing into paragraphs with line wrapping for readability
import textwrap 
w = textwrap.TextWrapper(replace_whitespace=False, break_long_words=False, width=60, initial_indent='   ', subsequent_indent='  ')
def see(text): print('\n\033[3m' + '\n\n'.join(['\n'.join(w.wrap(line))
                 for line in text.splitlines() if line.strip() != '']) + '\033[0m\n')

In [41]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTNeoConfig, LlamaForCausalLM
import torch

# model_name = 'GPT-9.0M-2L-4H-516C-1024I'
# model_name = 'BERT-9.0M-2L-4H-516C-1024I' # bert won't work for generation unless you fine-tune it for that task

tokenizer = AutoTokenizer.from_pretrained('10k-tok')
tokenizer.pad_token_id = tokenizer.eos_token_id


# Define the path where your model is saved
# custom_model = CustomGPTNeoForCausalLM.from_pretrained(f'results/models_baseline/{model_name}')
# custom_model

# print(type(custom_model))
# # Load the model configuration
config = GPTNeoConfig.from_pretrained(f'results/models_baseline/{model_name}')
# Initialize your custom model
custom_model = CustomGPTNeoForCausalLM(config_gqa=config)
custom_model.set_kqv_size(kqv_size)
custom_model.set_attention()

trained_state_dict = torch.load(f'results/models_baseline/{model_name}/model_state.pt')
custom_model.load_state_dict(trained_state_dict)

# print(custom_model.num_parameters())
#
# model = AutoModelForCausalLM.from_pretrained(f'results/models_baseline/{model_name}')
# state_dict = model.state_dict()
#
# custom_model.load_state_dict(state_dict)

9586688


#### Inference 
Let's generate a short story like the ones the model has been trained on! You'll notice that the prompt is surrounded by `<s>`, the begin-of-sequence (bos) token, and `</s>` end-of-sequence (eos) / separator (sep) token. This is from the BERT-style tokenisation, making it clear to the model where (one of several) input sequences ends.

In [42]:
import time
from memory_profiler import memory_usage

prompt = 'Once upon a time, there was a chicken with'
input_ids = tokenizer.encode(prompt, return_tensors='pt')

memory_usage_start = memory_usage(max_usage=True)

start_time = time.time()
output = custom_model.generate(input_ids, max_length=50, num_beams=1)
print("The output length is: ", len(output[0]))
end_time = time.time()

memory_usage_end = memory_usage(max_usage=True)

output_text = tokenizer.decode(output[0])

# textwrap with indentation on every new paragraph
see(output_text)
inference_time = end_time - start_time

print("Inference Time:", inference_time, "seconds")
print("Peak Memory Usage:", memory_usage_end, "MiB")

The output length is:  50

   <s>Once upon a time, there was a chicken with</s>.

Inference Time: 0.4837372303009033 seconds
Peak Memory Usage: 365.00390625 MiB


In [44]:
import time
from memory_profiler import memory_usage

# Define your prompt and encode it
prompt = 'Once upon a time, there was a chicken with'
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Set the number of times to generate output
num_iterations = 10  # You can adjust this number as needed

# Initialize a list to store inference times
inference_times = []
# Initialize a list to store peak memory usages per inference
memory_usage_values = []

# Loop for generating output multiple times
for _ in range(num_iterations):
    # Start recording memory to find the peak usage
    memory_usage_start = memory_usage(max_usage=True)

    # Record the start time
    start_time = time.time()
    # Generate the output
    output = custom_model.generate(input_ids, max_length=300, num_beams=1)
    # Record the end time
    end_time = time.time()

    # End recording memory to find the peak usage
    memory_usage_end = memory_usage(max_usage=True)

    # Calculate the time taken for inference
    inference_time = end_time - start_time
    inference_times.append(inference_time)

    memory_usage_values.append(memory_usage_end)

    # Decode and print the output text (optional)
    output_text = tokenizer.decode(output[0])
    print("Generated Text:", output_text)

average_inference_time = sum(inference_times) / num_iterations
average_memory_usage = sum(memory_usage_values) / num_iterations

print("Average Inference Time:", average_inference_time, "seconds")
print("Average Peak Memory Usage:", average_memory_usage, "MiB")

Generated Text: <s>Once upon a time, there was a chicken with</s>.































































































































































































































































































Generated Text: <s>Once upon a time, there was a chicken with</s>.































































































































































































































































































Generated Text: <s>Once upon a time, there was a chicken with</s>.



































































































































































































































In [92]:
prompt = 'Once upon a time, there was a chicken with'
input_ids = tokenizer.encode(prompt, return_tensors='pt')

output = custom_model.generate(
    input_ids,                              # input to the model
    max_length=300,                         # maximum generation length
    eos_token_id=tokenizer.eos_token_id,    # early stopping when eos_token is output

    # num_beams=1,                            # number of beams to use in generation
    temperature=1,
)
output_text = tokenizer.decode(output[0])

# textwrap with indentation on every new paragraph
see(output_text)
output


   <s>Once upon a time, there was a chicken with</s>



tensor([[   1, 7454, 2402,  257,  640,   11,  612,  373,  257, 9015,  351,    2,
          198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,
          198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,
          198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,
          198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,
          198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,
          198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,
          198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,
          198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,
          198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,
          198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,
          198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,  198,
          198,  198,  198,  

Sounds like me after a few beers too many, but at least the grammar is (mostly) correct. The model also learns some basic reasoning-like associations like being 'so high' allows you to see 'the whole world'. 